In [2]:
import sys,os
import numpy as np
import cupy as cp

from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator

os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='12' # set number of MKL threads to run in parallel

In [3]:
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2

I = np.eye(2)
I4 = np.eye(4)

In [4]:
SzSz = np.kron(Sz,Sz).real.astype(np.float64)
SxSx = np.kron(Sx,Sx).real.astype(np.float64)
SySy = np.kron(Sy,Sy).real.astype(np.float64)
o = np.kron(I, Sz) + np.kron(Sz, I)
Sp = (Sx + 1j*Sy).real
Sm = (Sx - 1j*Sy).real
G_ = np.kron(Sp, Sm) + np.kron(Sm, Sp)

In [5]:
G_2 = np.zeros((4,4))
G_2[0, 1] = G_2[1,0] = 1
G_2[2, 3] = G_2[3,2] = 0.5
G_2[0,2] = G_2[2,0] = -0.5
np.save("test2.npy", G_2)

In [6]:
L = 10
x = np.load("1D_symm.npy")
sys.path.append('../reduce_nsp')

In [7]:

from nsp.utils import l2nl

G2 = np.zeros((2**L, 2**L))
for bond in [[i, j] for i in range(L) for j in range(L) if i < j]:
    G2 += l2nl(G_2.reshape(4,4), L, bond , sps=2)
    
G2_gpu = cp.asarray(G2)

G3 = np.zeros((2**L, 2**L))
for bond in [[i, j] for i in range(L) for j in range(L) if i < j]:
    G3 += l2nl(x.reshape(4,4), L, bond , sps=2)
    
G3_gpu = cp.asarray(G3)

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
y = x.reshape(2,2,2,2).copy()
for i in range(2):
    y[:,i,:,i] = 0
    y[i,:,i,:] = 0    
G4 = np.zeros((2**L, 2**L))
for bond in [[i, j] for i in range(L) for j in range(L) if i < j]:
    G4 += l2nl(y.reshape(4,4).T, L, bond , sps=2)
    
G4_gpu = cp.asarray(G4)
np.save("1D_no_onesite.npy", y)


In [9]:
z = x.reshape(2,2,2,2).copy()
for i in range(2):
    for j in range(2):
        z[j,i,j,i] = 0
        z[i,j,i,j] = 0    
G5 = np.zeros((2**L, 2**L))
for bond in [[i, j] for i in range(L) for j in range(L) if i < j]:
    G5 += l2nl(z.reshape(4,4), L, bond , sps=2)
    
G5_gpu = cp.asarray(G5)
np.save("1D_no_diagonal.npy", z)

In [10]:
# o1 = y.reshape(4,4).copy()
# o1[0,3] = o1[3,0] = 0

# np.save("test.npy", o1)

In [11]:
a1 = np.zeros((4,4))
a1[1,2] = a1[2,1] = 1
a2 = np.zeros((4,4))
a2[3,0] = a2[0,3] = 1
a3 = np.zeros((4,4))
a3[3,0] = a3[0,3] = 1

In [12]:
E1 = l2nl(a1, L, [0,1] , sps=2)
# for bond in [[i, j] for i in range(L) for j in range(L) if i < j]:
#     G6 += l2nl(np.ones(16).reshape(4,4), L, bond , sps=2)
E1 = cp.asarray(E1)


E2 = l2nl(a2, L, [0,1] , sps=2)
E2 = cp.asarray(E2)
E3 = l2nl(a3, L, [0,1] , sps=2)
E3 = cp.asarray(E3)


In [13]:
Jz, Jx, Jy, hz, hx = [-1, 0.5, 0.3, 0, 0.5]
# Jz, Jx, Jy, hz, hx = [0,0,0, 0, 1]


J_zz = [[1,i,(i+1)%L] for i in range(L)]
J_xy = [[1,i,(i+1)%L] for i in range(L)]
h_list = [[-1.0,i] for i in range(L)]
ops_dict = dict(Jz=[["zz", J_zz]], # J for z
                Jx=[["xx",J_xy]], # J for x and y
                Jy=[["yy",J_xy]],
                hz=[["z",h_list]],
                hx=[["x",h_list]])
Mz_list = [[1.0/L,i] for i in range(L)]
# Mx_list = [[0.3/L,i] for i in range(L)]
G_list = [[1.0, i, j] for i in range(L) for j in range(L) if i < j]

basis_2d = spin_basis_1d(L ,pauli=False)
_H = quantum_operator(ops_dict,basis=basis_2d, dtype=np.float64, check_symm=False)
M = hamiltonian([["z",Mz_list]],[],basis=basis_2d, dtype=np.float64)
G = hamiltonian([["-+",G_list], ["+-",G_list]],[],basis=basis_2d, dtype=np.float64)
params_dict=dict(Jz=Jz, Jx=Jx,Jy=Jy,hz=hz, hx=hx)

Hermiticity check passed!
Hermiticity check passed!
Hermiticity check passed!


/tmp/ipykernel_21221/3182355799.py:19: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.basis_1d.spin.spin_basis_1d'>, to turn off this warning set check_symm=False in hamiltonian
  M = hamiltonian([["z",Mz_list]],[],basis=basis_2d, dtype=np.float64)
/tmp/ipykernel_21221/3182355799.py:20: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.basis_1d.spin.spin_basis_1d'>, to turn off this warning set check_symm=False in hamiltonian
  G = hamiltonian([["-+",G_list], ["+-",G_list]],[],basis=basis_2d, dtype=np.float64)


In [37]:
Hs = _H.tohamiltonian(params_dict)
H = Hs.toarray()
H_gpu = cp.asarray(H)

M_np = M.toarray()
M_gpu = cp.asarray(M_np)
G_gpu = cp.asarray(G.toarray())

In [38]:
E, V = cp.linalg.eigh(H_gpu)

In [39]:
T = 0.5
beta = 1/T
Z = cp.exp(-beta*E).sum()
M_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, M_gpu) / Z
M2_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, M_gpu@M_gpu) / Z
E_mean = (E*cp.exp(-beta*E)).sum() / Z
E_square_mean = ((E*E)*cp.exp(-beta*E)).sum() / Z
G_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, G_gpu) / Z
G2_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, G2_gpu) / Z
G3_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, G3_gpu) / Z
G4_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, G4_gpu) / Z
G5_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, G5_gpu) / Z
E1_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, E1) / Z
E2_mean = cp.einsum("n,in,jn,ij->", cp.exp(-beta*E), V, V, E2) / Z

In [40]:
print(f"L = {L}", params_dict)
print(f"T               = {T}")
print(f"E               = {E_mean / L}")
print(f"C               = {(E_square_mean - E_mean**2)*(beta**2)/L}")
print(f"M               = {M_mean}")
print(f"M^2             = {M2_mean}")
print(f"G               = {G_mean}")
print(f"G2(G)           = {G2_mean}")
print(f"G3              = {G3_mean}")
print(f"G4(No one site) = {G4_mean}")
print(f"G5(No diagonal) = {G5_mean}")
# print(f"phys cnt        = {1 / G6_mean}")
print(f"E1        = {E1_mean}")
print(f"E2        = {E2_mean}")

L = 10 {'Jz': -1, 'Jx': 0.5, 'Jy': 0.3, 'hz': 0, 'hx': 0.5}
T               = 0.5
E               = -0.17920077733642997
C               = 0.1904521933525169
M               = 2.2011215518437648e-17
M^2             = 0.060388084315426575
G               = 1.107024487276901
G2(G)           = 6.359046488914256
G3              = 28.46201476783561
G4(No one site) = 0.732466347939629
G5(No diagonal) = 18.489228471076085
E1        = -0.04242390087746677
E2        = 0.006167896186444027


In [ ]:
0.020723304513837117 / 0.605358

In [ ]:
8.464660399793908 / 3.72963

In [ ]:
0.17719472483205861 / 0.171871

In [ ]:
0.2728582784660241 / 0.250143

In [ ]:
0.3872213252980646 / 0.330939

In [ ]:
0.5208056763100141 / 0.412862

In [ ]:
0.6738376801266748 / 0.48966

In [ ]:
0.846417397441997 / 0.569147

In [ ]:
1.0385878074655466 / 0.643328

In [ ]:
1.2503667614679046 / 0.721431

In [ ]:
1.4817614530671108 / 0.789988 

In [ ]:
y = [1.030975119898404, 
 1.090809171018274,
 1.1700685784935128,
 1.2614521954309528,
 1.3761338073901785,
 1.487168336900655,
 1.6143985765667692,
 1.7331758151062326,
 1.875675900225207
]
y = np.array(y)
x = np.arange(4, 13)

In [ ]:
from matplotlib import pyplot as plt

# plt.yscale('log')
# plt.xscale('log')
plt.plot(x, y - 1)
